In [19]:
import requests
from glob import glob
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep
from tqdm import tqdm
import numpy as np
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [5]:
# url = 'https://www.amazon.com/Just-BARE-Chicken-Hand-Trimmed-Boneless/dp/B00AR0ZZ62/ref=sr_1_2?_encoding=UTF8&c=ts&dchild=1&keywords=Chicken&qid=1624295062&s=grocery&sr=1-2&ts_id=2256156011'
# # url ='https://www.amazon.com/Just-BARE-Chicken-Hand-Trimmed-Tenderloins/dp/B07PSZTSGH/ref=sr_1_3?_encoding=UTF8&c=ts&dchild=1&keywords=Chicken&qid=1624295062&s=grocery&sr=1-3&ts_id=2256156011'

# page = requests.get(url, headers=HEADERS)
# soup = BeautifulSoup(page.content, features="lxml")

# #product title
# title = soup.find(id='productTitle').get_text().strip()



# print(soup.find('span', class_ = 'a-size-small a-color-price'))


# # print(soup)
# # assert False

# # # print(soup.find(id='priceblock_ourprice'))
# # assert False
# try:
#     price = float(soup.find(id='priceblock_ourprice').get_text().replace('.', '').replace('$', '').replace(',', '.').strip())
# except:
#     # this part gets the price in dollars from amazon.com store
#     try:
#         price = float(soup.find(id='priceblock_saleprice').get_text().replace('$', '').replace(',', '').strip())
#     except:
#         price = 'NAN'

# print(title)
# print(price)

None
Just Bare Natural Fresh Chicken Breast Fillets | No Antibiotics Ever | Boneless | Skinless | 1.0 LB
NAN


($5.66 / lb)


In [20]:

url  = 'https://www.amazon.com/s?k=apple&i=wholefoods&ref=nb_sb_noss_2'

page = requests.get(url, headers = HEADERS)
soup = BeautifulSoup(page.content, features="lxml")

# print(soup)

def get_price(entry):
    return float(entry.split('/')[0].split('$')[1])

#product title
# title = soup.find(id='productTitle').get_text().strip()



souplist = [s.get_text() for s in soup.find_all('span', class_ = 'a-size-base a-color-secondary')]

newlist = []
for s in souplist:
    if 'lb' in s:
        newlist.append(s)

get_price(souplist[0])



IndexError: list index out of range

In [23]:
url = 'https://www.amazon.com/s?k=apples&i=wholefoods&ref=nb_sb_noss_1'

page = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(page.content, features="lxml")

l1 = [l.get_text() for l in soup.find_all('span', class_ = 'a-offscreen')]
l2 = [l.get_text() for l in soup.find_all('span', class_ = 'a-size-base a-color-base')]

print(l1)
print(l2[:len(l1)])



# print(soup)

[]
[]


In [15]:
productList = ['Bananas','Apples','Strawberries']

#the rule for deduction is basis value + profit margin/2
#in our case this means that we will take 


def scrapePricePerPound(productList,lbsDonated = 1,profitMargin = .2):
    datadict = {"Product":[],'Fair Market Price Per Pound':[], 'Basis Price Per Pound':[],
                'Pounds Donated':[], 'Deduction Expected':[]}
    
    
    if isinstance(lbsDonated, int) or isinstance(lbsDonated, float):
        donateList = False
    elif isinstance(lbsDonated, list):
        
        assert len(lbsDonated) == len(productList), 'List lengths do not match'
        donateList = True
        
    print('Calculating Deductions For Simulated Donations')
    for i in tqdm(range(len(productList))):
        
        p = productList[i]
        url = 'https://www.amazon.com/s?k='+p+'&i=wholefoods&ref=nb_sb_noss_1'
        
        page = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(page.content, features="lxml")
        
        try:
            l1 = [l.get_text() for l in soup.find_all('span', class_ = 'a-offscreen')]
            l2 = [l.get_text() for l in soup.find_all('span', class_ = 'a-size-base a-color-base')]

            l2 = l2[:len(l1)]
            
            print(l1)
            print(l2)
            assert False
            pricefound = False
            
            
            for j in range(len(l1)):
                if 'lb' in l2[j]:
                    price = float(l1[j].replace('$',''))
                    pricefound = True
                    
                    break
            if not pricefound:
                
                assert False
        except:
            #do other system
            souplist = [s.get_text() for s in soup.find_all('span', class_ = 'a-size-base a-color-secondary')]
            
            for s in souplist:
                if 'lb' in s:
                    price = s.split('/')[0].replace('$','').replace('(','')
                    price = float(price)
                    break
                elif 'Fl Oz' or 'Ounce' in s:
                    price = 16*float(s.split('/')[0].replace('$','').replace('(',''))
                    break
                    
                else:
                    price = np.nan
                    
        basisValue = round(price/(1+profitMargin),2)
        datadict['Basis Price Per Pound'].append(basisValue)
        deductionRate = basisValue + (profitMargin/2.)*basisValue
        
        if donateList:
#             print(i)
#             print(lbsDonated[i])
            
            deduction = float(lbsDonated[i])*deductionRate
            
            datadict['Pounds Donated'].append(float(lbsDonated[i]))
            
        else:
            deduction = float(lbsDonated)*deductionRate
            datadict['Pounds Donated'].append(float(lbsDonated))
            
        datadict['Deduction Expected'].append(round(deduction,2))
                    
#         print('Price of '+p+' is '+str(price)+'/lb')
        datadict['Product'].append(p)
        datadict['Fair Market Price Per Pound'].append(price)
    datadict['Product'].append('Total')
    datadict['Pounds Donated'].append(sum(datadict['Pounds Donated']))
    datadict['Deduction Expected'].append(sum(datadict['Deduction Expected']))
    datadict['Basis Price Per Pound'].append(0)
    datadict['Fair Market Price Per Pound'].append(0)
    total_deduction = sum(datadict['Deduction Expected'])
    print('Total deduction expected is '+str(total_deduction))
    print('Deduction expected using old system is '+str(1.57*sum(datadict['Pounds Donated'])))
    return datadict

# scrapePricePerPound(productList)  
                
# df= pd.read_csv('producelist.csv')
# productList = list(df['Product Name'].values)
# lbsDonated = list(50*np.random.rand(len(productList)))


# print(isinstance)

# datadict = 

# print()

df = pd.DataFrame(scrapePricePerPound(productList))
df.head()
# df.to_csv('deductionsimulatortest.csv',index = False)

  0%|          | 0/3 [00:00<?, ?it/s]

Calculating Deductions For Simulated Donations


 33%|███▎      | 1/3 [00:00<00:01,  1.30it/s]

['$0.69', '$1.99', '$0.59', '$4.99', '$2.99', '$3.19', '$5.99', '$3.69', '$1.29', '$12.59', '$6.99', '$2.29', '$7.99', '$4.22', '$6.29', '$4.22', '$6.29', '$1.49', '$3.69', '$4.99', '$2.69', '$7.99', '$3.15', '$4.99', '$6.79', '$3.69', '$1.49']
['/lb', '/lb', '/lb', '/lb', '/lb', ' or ', 'Prime Day Deals', 'Any Department', 'Whole Foods Market', 'Baby Foods', 'Beverages', 'Breads & Bakery', 'Breakfast Foods', 'Candy & Chocolate', 'Dairy, Cheese & Eggs', 'Deli & Prepared Foods', 'Frozen', 'Meat & Seafood', 'Pantry Staples', 'Produce', 'Snack Foods', '365 by Whole Foods Market', 'Happy Baby', 'Simple Mills', 'GoGo SqueeZ', 'Amazing Grass', 'Barnana']


 67%|██████▋   | 2/3 [00:01<00:00,  1.30it/s]

['$2.99', '$5.99', '$2.49', '$5.99', '$2.99', '$3.99', '$8.99', '$2.49', '$21.59', '$29.99', '$5.99', '$3.32', '$4.29', '$2.49', '$6.79', '$9.89', '$11.79', '$12.59', '$2.99', '$8.29', '$4.19', '$11.99', '$2.99', '$6.99', '$9.79', '$1.99', '$6.69']
['/lb', '/lb', '/lb', '/lb', '/lb', '/lb', ' or ', 'Prime Day Deals', 'Any Department', 'Whole Foods Market', 'Baby Foods', 'Beverages', 'Breads & Bakery', 'Breakfast Foods', 'Candy & Chocolate', 'Dairy, Cheese & Eggs', 'Deli & Prepared Foods', 'Food & Beverage Gifts', 'Frozen', 'Meat & Seafood', 'Meat Substitutes', 'Pantry Staples', 'Produce', 'Snack Foods', '365 by Whole Foods Market', 'Applegate', 'Whole Foods Market']


100%|██████████| 3/3 [00:02<00:00,  1.25it/s]

['$6.49', '$3.69', '$4.99', '$7.49', '$4.99', '$2.99', '$4.69', '$4.99', '$1.99', '$4.04', '$4.99', '$3.59', '$4.99', '$5.79', '$4.99', '$3.99', '$4.04', '$4.99', '$6.99', '$4.49', '$4.99', '$5.39', '$6.99', '$7.99', '$4.99', '$1.99', '$3.49', '$6.99']
[' or ', 'Prime Day Deals', 'Any Department', 'Whole Foods Market', 'Baby Foods', 'Alcoholic Beverages', 'Beverages', 'Breads & Bakery', 'Breakfast Foods', 'Candy & Chocolate', 'Dairy, Cheese & Eggs', 'Food & Beverage Gifts', 'Frozen', 'Meat & Seafood', 'Pantry Staples', 'Produce', 'Snack Foods', '365 by Whole Foods Market', 'Whole Foods Market', 'Stonyfield Organic', 'Forager Project', 'Crofters', 'Icelandic Provisions', "Annie's Homegrown", 'Under $10', '$10 to $20', '$20 to $30', '$30 to $50']
Total deduction expected is 44.56
Deduction expected using old system is 9.42


,Product,Fair Market Price Per Pound,Basis Price Per Pound,Pounds Donated,Deduction Expected
0,Bananas,19.04,15.87,1.0,17.46
1,Apples,2.00,1.67,1.0,1.84
2,Strawberries,3.25,2.71,1.0,2.98
3,Total,0.00,0.00,3.0,22.28


In [92]:
import tkinter as tk
